In [1]:
from recommender.querying import CHROMA_Manager
import recommender.api
from recommender.api import db
from recommender.querying.sql_models import Movie
from recommender import REPO_PATH
from recommender.python_queue.job import generate_recommendations
from recommender.recommenders import CombinedRecommender
import json
import optuna

/Users/fhammer/miniconda3/envs/reco/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open(REPO_PATH / "rating_movies.json") as f:
    rating_movies = json.load(f)


EXAMPLE_MOVIES = Movie.query.filter(Movie.title.in_(rating_movies)).all()

MY_RATINGS = [4,2,5,1,5,4,3,2]

[i.title for i in EXAMPLE_MOVIES]

['Gladiator',
 'Jurassic Park',
 'Memento',
 'Pretty Woman',
 'Pulp Fiction',
 'Shawshank Redemption, The',
 'Star Wars: Episode IV - A New Hope',
 'Toy Story']

In [3]:
movies_i_like = [
    "Blade Runner",
    "Inception",
    "Fight Club",
    "Old Boy," "American Beauty",
    "Fargo",
    "Interstellar",
    "Batman Begins",
    "Silence of the Lambs, The",
    "Schindler's List",
]

In [4]:
weights = {
    "user": 1.2,
    "content": 4,
    "user_weights": {
        "default_vec_val": 2.5,
        "closest_relevant_users": 5,
    },
    "content_weights": {
        "directors": 1,
        "actors": 0.2,
        "plots": 1.2,
    }
}

In [5]:

def objective(trial : optuna.trial):
    
    user = trial.suggest_float('user', 0.2, 120)
    content = trial.suggest_float('content', 0.2, 20)
    closest_relevant_users = trial.suggest_int('closest_relevant_users', 3, 13)
    directors = trial.suggest_float('directors', 0.2, 10)
    actors = trial.suggest_float('actors', 0.2, 10)
    plots = trial.suggest_float('plots', 0.2, 10)
    
    
    CombinedRecommender.weights["user"] = user
    CombinedRecommender.weights["content"] = content
    CombinedRecommender.weights["content_weights"] =  {
        "directors": directors,
        "actors":  actors,
        "plots":  plots,
    }
    CombinedRecommender.weights["user_weights"]["closest_relevant_users"] = closest_relevant_users

    recommender = CombinedRecommender(db)
    recommendations = recommender.find_most_simmilar(EXAMPLE_MOVIES,MY_RATINGS)
    
    loss = 0
    
    for i,movie in enumerate(recommendations[0][:50]):
        
        try:
            og_pos = movies_i_like.index(movie.title)
            print("found ", movie.title)
            og_pos_weight = (len(movies_i_like) - og_pos) * 2
            
            reco_weigth = (50 - i)**3
            print(og_pos_weight, reco_weigth, i , og_pos)
            loss -= reco_weigth * og_pos_weight


        except ValueError:
            continue

    return loss


In [6]:

study = optuna.create_study()
study.optimize(objective, n_trials=50)

[I 2023-12-22 17:49:13,352] A new study created in memory with name: no-name-01785c66-0dd3-42b5-b4b5-ba72ccb8f871
2023-12-22 17:49:24.687 | DEBUG    | recommender.recommenders.combined_recommender:find_most_simmilar:40 - User Based Recommendation took 11.332072019577026
2023-12-22 17:49:25.019 | INFO     | recommender.recommenders.content_based_recommender:get_scores:88 - direcor 0.3319518566131592
2023-12-22 17:49:30.748 | INFO     | recommender.recommenders.content_based_recommender:get_scores:105 - actors 5.7277209758758545
2023-12-22 17:49:31.042 | INFO     | recommender.recommenders.content_based_recommender:get_scores:114 - plots 0.29327988624572754
2023-12-22 17:49:31.043 | INFO     | recommender.recommenders.content_based_recommender:find_most_simmilar:34 - the scores 6.3550450801849365
2023-12-22 17:49:31.043 | DEBUG    | recommender.recommenders.combined_recommender:find_most_simmilar:46 - Content Based Recommendation took 6.355993032455444
[I 2023-12-22 17:49:31,049] Trial 0

found  Fargo
10 110592 2 4
found  Fight Club
14 85184 6 2
found  Silence of the Lambs, The
4 50653 13 7
found  Schindler's List
2 39304 16 8


2023-12-22 17:49:34.533 | DEBUG    | recommender.recommenders.combined_recommender:find_most_simmilar:40 - User Based Recommendation took 3.4835410118103027
2023-12-22 17:49:34.866 | INFO     | recommender.recommenders.content_based_recommender:get_scores:88 - direcor 0.3316061496734619
2023-12-22 17:49:40.414 | INFO     | recommender.recommenders.content_based_recommender:get_scores:105 - actors 5.547663927078247
2023-12-22 17:49:40.711 | INFO     | recommender.recommenders.content_based_recommender:get_scores:114 - plots 0.2960519790649414
2023-12-22 17:49:40.711 | INFO     | recommender.recommenders.content_based_recommender:find_most_simmilar:34 - the scores 6.177325010299683
2023-12-22 17:49:40.712 | DEBUG    | recommender.recommenders.combined_recommender:find_most_simmilar:46 - Content Based Recommendation took 6.178447246551514
[I 2023-12-22 17:49:40,718] Trial 1 finished with value: -2090842.0 and parameters: {'user': 72.67173420341939, 'content': 19.837042535725278, 'closest_

found  Fargo
10 103823 3 4
found  Fight Club
14 54872 12 2
found  Silence of the Lambs, The
4 32768 18 7
found  Schindler's List
2 29791 19 8
found  Batman Begins
6 15625 25 6


2023-12-22 17:49:52.539 | DEBUG    | recommender.recommenders.combined_recommender:find_most_simmilar:40 - User Based Recommendation took 11.820337057113647
2023-12-22 17:49:52.887 | INFO     | recommender.recommenders.content_based_recommender:get_scores:88 - direcor 0.34772562980651855
2023-12-22 17:49:58.614 | INFO     | recommender.recommenders.content_based_recommender:get_scores:105 - actors 5.725843906402588
2023-12-22 17:49:58.918 | INFO     | recommender.recommenders.content_based_recommender:get_scores:114 - plots 0.3036198616027832
2023-12-22 17:49:58.919 | INFO     | recommender.recommenders.content_based_recommender:find_most_simmilar:34 - the scores 6.379108905792236
2023-12-22 17:49:58.920 | DEBUG    | recommender.recommenders.combined_recommender:find_most_simmilar:46 - Content Based Recommendation took 6.3800389766693115
[I 2023-12-22 17:49:58,925] Trial 2 finished with value: -2544650.0 and parameters: {'user': 31.526032191567968, 'content': 7.139754846959873, 'closes

...

found  Batman Begins
6 103823 3 6
found  Blade Runner
18 2744 36 0
found  Inception
16 27 47 1


2023-12-22 18:18:59.716 | DEBUG    | recommender.recommenders.combined_recommender:find_most_simmilar:40 - User Based Recommendation took 4.771611213684082
2023-12-22 18:19:00.121 | INFO     | recommender.recommenders.content_based_recommender:get_scores:88 - direcor 0.4008500576019287
2023-12-22 18:19:06.349 | INFO     | recommender.recommenders.content_based_recommender:get_scores:105 - actors 6.227006196975708
2023-12-22 18:19:06.805 | INFO     | recommender.recommenders.content_based_recommender:get_scores:114 - plots 0.45146989822387695
2023-12-22 18:19:06.807 | INFO     | recommender.recommenders.content_based_recommender:find_most_simmilar:34 - the scores 7.087383031845093
2023-12-22 18:19:06.810 | DEBUG    | recommender.recommenders.combined_recommender:find_most_simmilar:46 - Content Based Recommendation took 7.090773820877075
[I 2023-12-22 18:19:06,828] Trial 48 finished with value: -2844466.0 and parameters: {'user': 33.31987430105625, 'content': 8.621882203892284, 'closest_

found  Fight Club
14 125000 0 2
found  Fargo
10 85184 6 4
found  Schindler's List
2 35937 17 8
found  Silence of the Lambs, The
4 21952 22 7
found  Batman Begins
6 13824 26 6


2023-12-22 18:19:09.295 | DEBUG    | recommender.recommenders.combined_recommender:find_most_simmilar:40 - User Based Recommendation took 2.4503719806671143
2023-12-22 18:19:09.662 | INFO     | recommender.recommenders.content_based_recommender:get_scores:88 - direcor 0.36614084243774414
2023-12-22 18:19:15.980 | INFO     | recommender.recommenders.content_based_recommender:get_scores:105 - actors 6.316848278045654
2023-12-22 18:19:16.315 | INFO     | recommender.recommenders.content_based_recommender:get_scores:114 - plots 0.33469390869140625
2023-12-22 18:19:16.316 | INFO     | recommender.recommenders.content_based_recommender:find_most_simmilar:34 - the scores 7.019846200942993
2023-12-22 18:19:16.317 | DEBUG    | recommender.recommenders.combined_recommender:find_most_simmilar:46 - Content Based Recommendation took 7.0210161209106445
[I 2023-12-22 18:19:16,329] Trial 49 finished with value: -3199978.0 and parameters: {'user': 68.17031766532608, 'content': 3.0858939912480023, 'clos

found  Fight Club
14 125000 0 2
found  Fargo
10 117649 1 4
found  Silence of the Lambs, The
4 54872 12 7
found  Schindler's List
2 27000 20 8


In [7]:
study.best_params

{'user': 31.696782262221433,
 'content': 15.069030305424409,
 'closest_relevant_users': 4,
 'directors': 9.053375597465866,
 'actors': 1.2301894300715557,
 'plots': 2.406376778502906}

In [8]:
weights = {
    "user": 31.696782262221,
    "content": 15.069030305424,
    "user_weights": {
        "default_vec_val": 2.5,
        "closest_relevant_users": 4,
    },
    "content_weights": {
        "directors":  9.0533755974,
        "actors": 1.2301894300,
        "plots": 2.406376778502906,
    }
}

CombinedRecommender.weights = weights
recommender = CombinedRecommender(db)
recommendations = recommender.find_most_simmilar(EXAMPLE_MOVIES,MY_RATINGS)

2023-12-22 18:20:47.727 | DEBUG    | recommender.recommenders.combined_recommender:find_most_simmilar:40 - User Based Recommendation took 12.935786008834839
2023-12-22 18:20:48.080 | INFO     | recommender.recommenders.content_based_recommender:get_scores:88 - direcor 0.35257577896118164
2023-12-22 18:20:53.954 | INFO     | recommender.recommenders.content_based_recommender:get_scores:105 - actors 5.872028350830078
2023-12-22 18:20:54.279 | INFO     | recommender.recommenders.content_based_recommender:get_scores:114 - plots 0.3236210346221924
2023-12-22 18:20:54.279 | INFO     | recommender.recommenders.content_based_recommender:find_most_simmilar:34 - the scores 6.551568984985352
2023-12-22 18:20:54.280 | DEBUG    | recommender.recommenders.combined_recommender:find_most_simmilar:46 - Content Based Recommendation took 6.5525689125061035


In [9]:
[i.title for i in recommendations[0][:20]]

['Reservoir Dogs',
 'Fight Club',
 'Green Mile, The',
 'Fargo',
 'Star Wars: Episode VI - Return of the Jedi',
 'Star Wars: Episode V - The Empire Strikes Back',
 'Goodfellas',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark)',
 'Matrix, The',
 'Django Unchained',
 'Inception',
 'Princess Bride, The',
 'Usual Suspects, The',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb',
 'Being John Malkovich',
 'Batman Begins',
 'Dunkirk',
 'Toy Story 2',
 'Rob Roy',
 'Fugitive, The']